# Using *Key Point Summarization* for analyzing and finding insights in a survey data 
When you have a large collection of texts representing people’s opinions (such as product reviews, survey answers or social media), it is difficult to understand the key issues that come up in the data. Going over thousands of comments is prohibitively expensive.  Existing automated approaches are often limited to identifying recurring phrases or concepts and the overall sentiment toward them, but do not provide detailed or actionable insights. 

In this tutorial you will gain hands-on experience in using *Key Point Summarization* (KPS) for analyzing and deriving insights from open-ended comments.  

The data we will use is [a community survey conducted in the city of Austin](https://data.austintexas.gov/dataset/Community-Survey/s2py-ceb7). In this survey, the citizens of Austin were asked "If there was ONE thing you could share with the Mayor regarding the City of Austin (any comment, suggestion, etc.), what would it be?". 

## 1. Initialization

### 1.1 Setup

Let's first import all the required packages for this tutorial and initialize the *Key Point Summarization* client. The client prints information using a logger and a suitable verbosity level should be set. The client object is configured with an API key. To receive an API key please send an email to *yoavka@il.ibm.com* and we'll be happy to provide it. In the code below it is stored in the enviroment variable *KPS_API_KEY* (you may also modify the code and place the api-key directly).

In [ ]:
from debater_python_api.api.clients.keypoints_client import KpsClient, KpsJobFuture
from debater_python_api.api.clients.key_point_summarization.KpsResult import KpsResult
import os
import pandas as pd
import json 

pd.set_option('display.max_rows', None)
KpsClient.init_logger()
api_key = os.environ['KPS_API_KEY']
host = 'https://keypoint-matching-backend.debater.res.ibm.com'
keypoints_client = KpsClient(api_key, host)

### 1.2 Read the data
Let's read the data from *dataset_austin.csv* file, which holds the Austin survey dataset, and print the first comment.

In [ ]:
comments_df = pd.read_csv('./dataset_austin.csv')

print(f'There are {len(comments_df)} comments in the dataset')
print(dict(comments_df.iloc[0,:]))

Each comment has a unique_id 'id', a 'text', a 'year' and a 'Council_District'. The *Key Point Summarization* service is able to run over hundreds of thousands of comments. However, to provide good user experience, the KPS evaluation service is limited to 1000 comments per run, each comment with up to 3000 chars. You may request to increase this limit if needed. 

We will choose a sample of 1000 comments from 2016 for our analysis.

In [ ]:
comments_df = comments_df.dropna()
comments_df = comments_df[comments_df.text.apply(lambda x: 0<len(str(x))<=3000)]

comments_2016_df = comments_df[comments_df['year'] == 2016]
sample_size = 1000
comments_2016_sample_df = comments_2016_df.sample(n = sample_size, random_state = 1)

## 2. Run the full KPS flow and generate results
The simplest way to run KPS is using the method **keypoints_client.run_full_kps_flow()**, which serves as an excellent starting point. To do so, you only need to provide a collection of textual comments and a distinct domain name (comprised of alphanumeric characters, spaces, or underscores). If you wish to reuse a domain from a previous run, you will first need to delete it via **keypoints_client.delete_domain_cannot_be_undone(domain=domain)**. 

Using this method, KPS extracts the key points from the provided data and matches each sentence in the input comments with its corresponding matching key points.

By default, the service performs stance analysis: it runs for positive (pro) and for negative (con) sentences seperately, and returns a merged result containing key points from both stances. To disable the stance analysis and run on all sentences together, add the parameter **stance=no-stance** to the **run_full_kps_flow** method.   

In [ ]:
domain = "austin_demo_full"
comments_texts_2016 = list(comments_2016_sample_df['text'])
kps_result_2016 = keypoints_client.run_full_kps_flow(domain, comments_texts_2016)

kps_result_2016 is a KpsResult object. Let's print the top 40 key points, and the top three matched sentences per key point:

In [ ]:
kps_result_2016.print_result(n_sentences_per_kp = 3, title = "Austin sample 2016 full flow", n_top_kps = 40)

In the second line we can see that 67% of the comments were matched, i.e., had at least one sentence matched to a key point.
For each key point, this method prints the number of matched comments, its stance (pro or con) and the top matching sentences which present some of main points that were raised regarding the key point.

## 3. KpsResult Object and result processing


The *KpsResult* object stores all the information about the job and the results. It can be used to generate several types of reports and to compare different results. All the reports generated in this section are available in the folder "kps_results".

The KpsResult can be saved to a file and loaded from a file via the **load** and **save** methods:

In [ ]:
json_file = "kps_results/kps_result_2016.json"
kps_result_2016.save(json_file) 
kps_result_2016 = KpsResult.load(json_file)

### 3.1 Result Summary
The result summary presented in the attribute *summary_df* displays the aggregated information per key point:

In [ ]:
kps_result_2016.summary_df

The report displays:  
- `key_point`: the list of generated key points, sorted by their saliance.
- `#comments`: the number of comments matched to the key point (comments that have at least one sentence matched to the key point).  
- `comments_coverage`: the percentage of comments matched to it (out of the entire set of comments sent to the job).  
- `#sentences`: the number of the sentences matched to the key point.  
- `sentences_coverage`: the percentage of the sentences matched to the key point (out of the entire set of sentences in the comments sent to the job).
- `stance`: the key point's stance (if stance analysis was performed).
- `kp_id`: the key point's id.  
- `parent_id`: The analysis also creates a key point tree-structured hierarchy. The parent_id column shows the kp_id of the parent of the key point. For example, the key points *Please plan better for growth on our roadways!* and *driving in Austin is terrible* are under the parent key point *Improve traffic flow.*  
- `n_comments_subtree`: how many comments are in the subtree of the key point (matched to the key point or any of its descendants).  


In addition to the individual key points, in the last rows we can find the statistics of total and matched number of sentences and comments for each stance, starting with \*: in the current example, there are 1000 comments and 1813 sentences in total, 668 and 920 of them are matched to at least one key point, respectively. Out of those, 881 comments have sentences classified as con, and overall 1450 sentences are classified as con. 655 comments and 898 sentences are matched to con key points.

### 3.2 Docx report
You can also generate a Microsoft Word document that shows the key point hierarchy visually and presents the sentences matched to each key point as a user-friendly report. it can be generated as follows:

In [ ]:
kps_result_2016.generate_docx_report(output_dir = "kps_results", result_name="kps_result_2016")

Note that this report displays only the more salient key points (that are matched to at least 5 sentneces) so there might be some differences from the summary report. 

### 3.3 Full report

The attribute *result_df* stores the full results. Each row stores a pair of a key point, a matched sentence, their match_score and all the information regarding the sentence.

In [ ]:
kps_result_2016.result_df.head(3)

It's also possible to generate all three reports together. You need to provide the output_dir and the result name, and the reports will be written to files with the appropriate suffixes:

In [ ]:
kps_result_2016.export_to_all_outputs(output_dir="kps_results", result_name="kps_result_2016")

More advanced capabiliteis of the KpsResult Object will be presented later: 
 - Job management related options (Section 5). 
 - Comparative analysis (Section 6).

## 4 Run KPS step by step
In order to customize the key points summarization service and fully exploit its caching and comparitive capabilities, we must run the service in a staged manner. Let's dive into each step and understand the KPS flow.  

### 4.1 Create a domain
The *Key Point Summarization* service stores the data (and cached results) in a *domain*. A user can create several domains, one for each dataset. Domains are only accessible to the user who created them.

Create a domain using the **keypoints_client.create_domain(domain=domain, domain_params={})** method. 
Several parameters can be passed in the domain_params dictionary. In most cases, the default params need not change, and provide satisfactory results. Full documentation of the supported *domain_params* can be found [here](kps_parameters.pdf).

By default, an exception is raised if the domain already exists. To avoid this exception, add the parameter **ignore_exists=True** to the method. Note that in this case the domain_params are not updated and the existing domain remains the same. 

In this tutorial we will first delete the domain to make sure that we start with an empty domain.

In [ ]:
domain = 'austin_demo'
keypoints_client.delete_domain_cannot_be_undone(domain=domain)
keypoints_client.create_domain(domain=domain, domain_params={})

Notes:
* The domain must be comprised of alphanumeric characters, spaces, or underscores. 
* We can delete a domain we no longer need using: **keypoints_client.delete_domain_cannot_be_undone(domain=domain)**.
* Each domain has a state: it stores all comments previously uploaded into it and a cache with all the computations performed over this data.
* If we want to restart and run over the domain from scratch (no comments and no cache), we can delete the domain and then re-create it. Keep in mind that the cache is also cleared and consecutive runs will take longer.

### 4.2 Upload comments into the domain
Upload the comments into the domain using the **keypoints_client.upload_comments(domain=domain, comments_ids=comments_ids, comments_texts=comments_texts)** method. This method receives the domain, a list of comment_ids and a list of comment_texts. When uploading comments into a domain, the *Key Point Summarization* service splits the comments into sentences and runs a minor cleansing on them. If you want to split the comments into sentences or clean them yourself, you can use the *split_comments* or *clean_comments* domain_params when creating the domain to disable this functionality in KPS (see details [here](kps_parameters.pdf)).

Note that:
* Comments_ids must be unique strings comprised of alphanumeric characters, spaces or underscores.
* The number of comments_ids must match the number comments_texts
* Comments_texts must not be longer than 3000 characters
* Uploading the same comment several times (same domain + comment_id + comment_text) is not a problem and the comment is only processed once.
* Uploading the same comment_id with a different comment_text will raise an exception. 

After being uploaded to the domain, the comments can take some time to be processed. 
The method runs in a synchronous manner and returns only after all the comments are processed. In the meantime, the status of the processing is printed on screen.
For information about running KPS asynchronously, see Section 7.

In [ ]:
comments_texts_2016 = list(comments_2016_sample_df['text'])
comments_ids_2016 = list(comments_2016_sample_df['id'].astype(str))
keypoints_client.upload_comments(domain=domain, comments_ids=comments_ids_2016, comments_texts=comments_texts_2016)

To examine the processed data, we can download the processed sentences and save them into a csv:

In [ ]:
sentences_df = keypoints_client.get_sentences_for_domain(domain=domain)
sentences_df.to_csv(f"kps_results/{domain}_sentences.csv")

### 4.3 Run a KPS job
Run a *Key Point Summarization* job using the **keypoints_client.run_kps_job(domain=domain)** method. 
This method receives:
* The domain.
* Optional *comment_ids*: by default, the summarization is performed over all comments in the domain. If we need to run over a subset of the comments (split the data by different GEOs/users types/timeframes etc.) we can pass a list of their comments_ids.
* Optional *run_params*: a dictionary with various parameters for customizing the job (see Section 4.4).
* Optional *stance*: unlike in the run_full_kps_flow method presented in Section 2, here no stance analysis is performed by default. see Section 4.5 for stance customization.
* Optional *description*: a description of the job to appear in the user report (see Section 5.1).  

The system extracts the key points from the input comments, and matches each sentence in the comments with all its matching key points.
The job runs in a synchronous manner, prints the progress to the screen and returns the KpsResult eventually.   
For information about running KPS asynchronously, see Section 7. 

In [ ]:
kps_result_2016_no_stance = keypoints_client.run_kps_job(domain = domain)

Let's print the top 20 key points in the results: 

In [ ]:
kps_result_2016_no_stance.print_result(n_sentences_per_kp = 3, title = "Austin sample 2016", n_top_kps = 20)

### 4.4 Modify the run_params to customize the summary
Each domain has a cache that stores all the intermediate results that are calculated during the summarization process. Therefore modifing the run_params and running another summarization is usually faster. 

Full documentation of the supported *run_params* can be found [here](kps_parameters.pdf).
Some of the notable options:
* By default, key points are extracted automatically. When we want to provide the key points and match all the sentences to them we can pass key_points parameter: **run_param['key_points'] = [...]**. This enables a mode of work named human-in-the-loop where we first automatically extract key points, then we manually edit them (refine non-perfect key points, remove duplicated and add missing ones) and then run again, this time providing the edited keypoints as a given set of key points.
* It is also possible to provide key points and let KPS add additional missing key points. To do so pass the key points to the key_point_candidates parameter: **run_param['key_point_candidates'] = [...]** (see Section 6.2 for an elaborated example).
* Change the lengths of the required key points and the sentences participating in the summarization.
* The **mapping_policy** is used when mapping all sentences to the final key points: the default value is **NORMAL**. Changing to **STRICT** will cause only the sentence and key point pairs with very high matching confidence to be considered matched, increasing precision but potentially decreasing coverage. Changing it to **LOOSE** will do the opposite and match pairs with lower confidene. 

Let's run with the 'LOOSE' mapping policy:

In [ ]:
run_params = {'mapping_policy':'LOOSE'}
kps_result_loose = keypoints_client.run_kps_job(domain=domain, run_params=run_params)
kps_result_loose.print_result(n_sentences_per_kp=3, title='Austin sample 2016 LOOSE', n_top_kps=20)

By changing the mapping policy to **LOOSE** the comments' coverage was increased from 74.5% to 82%.

### 4.5 Add stance analysis to the summarization

In many usecases (surveys, customer feedback, etc.) the comments have positive and/or negative stances, and it is useful to create a KPS on each stance seperatly. Most stance detection models don't perform too well on survey data since the comments have many "suggestions" in them. These suggestions tend to be classified by the models as positives, while the user suggests a point for improvement. For that end we trained a stance-model that handles suggestions well and classifies each sentence as either 'Positive', 'Negative', 'Neutral' or 'Suggestion'. We treat Suggestions as negatives and run two separate summarizations, first over 'Positive' sentences (pro) and second over 'Negative' and 'Suggestions' sentences (con).

This has the following advantages:

* Generate separate positive/negative key points that show clearly what works well and what needs to be improved.
* Filters-out neutral sentences that usually don't contain valuable information.
* Helps the matching model avoid stance mistakes (matching a positive sentence to a negative key point and vice-versa).

In some cases, we might want to run over a single stance. For example, if we are only interested in points for improvement.
In order to run for each stance seperately, use the **stance** parameter in **run_kps_job**. the options are either "pro", "con", or "no-stance" (default). For example, to run only on the "pro" sentences, run **keypoints_client.run_kps_job(domain=domain, stance="pro")**

To generate a merged pro and con result use the **run_kps_job_both_stances** method.
This method starts two seperate jobs simultenously, one for *pro* and one for *con*. It later unifies the results and returns the merged result object (similar to the default behviour of **run_kps_full_flow**).
This method receives: 
* The domain.
* comments_ids - optional, as in *run_kps_job*.
* desription - optional, as in *run_kps_job*, stance is appended to the description of each job.
* run_params_pro - optional run_params to be sent to the *pro* job.
* run_params_con - optional run_params to be sent to the *con* job.

In [ ]:
kps_result_2016_merged = keypoints_client.run_kps_job_both_stances(domain)

## 5. Jobs management

### 5.1 User report
The user report stores all the information about existing domains and all past and present KPS jobs. 
To fetch it and print to screen:

In [ ]:
report = keypoints_client.get_full_report()
keypoints_client.print_report(report)

### 5.2 job_id
Each job has a unique job_id which is useful for the jobs managements, and can be obtained in several ways:
* It's printed to the screen when the job starts and in every progress update.
* From the user report.
* When running asyncronously (see Section 7)
* From the KpsResult object, using the **get_stance_to_job_id()** method. Note that the KpsResult can either store the information from a single job (when running on a single stance or no stance) or two jobs (for combined pro and con results). 

In [ ]:
print(kps_result_2016_merged.get_stance_to_job_id())

In [ ]:
print(kps_result_2016_no_stance.get_stance_to_job_id())

### 5.3 Canceling a job
Simply exiting the program after the job is sent does not cancel the job: it keeps running on the server, consuming resources. In order to cancel a job, use: 
* **keypoints_client.cancel_kp_extraction_job(\<job_id\>)**

It is also possibe to stop all jobs in a domain, or even all jobs in all domains (this might be simpler since there is no need of the job_id):

* **keypoints_client.cancel_all_extraction_jobs_for_domain(\<domain\>)**
* **keypoints_client.cancel_all_extraction_jobs_all_domains()**


### 5.4 Fetching the results of a previous job
If the program terminated unexpectedly after the job was sent, you can still fetch the results using:
* **kps_result = keypoints_client.get_results_from_job_id(\<job_id\>)**

### 5.5 Fetching unmatched sentences
The KpsResult object stores only the sentences that were matched to at least one key point. In order to get the list of sentences that were not matched to any key point, the client needs to be called with the method **get_unmapped_sentences_for_kps_result(\<kps_result\>)**:

In [ ]:
unmapped_sentences_df = keypoints_client.get_unmapped_sentences_for_kps_result(kps_result_2016_no_stance)
unmapped_sentences_df.to_csv("kps_results/kps_result_2016_merged_unmapped_sentences.csv")

Note that this will work only if the domain on which the job(s) ran still exists, and thus is not supported when using the **run_full_kps_flow** method. 

## 6. Comparative analysis with KPS

The KPS service allows us to easily perform comparisons between subsets of our data and to perform trend analysis over data collected in different times. Let's explore two of these options: 

### 6.1 Compare comment subsets
So far, we ran over all the sampled comments for 2016. Now, let's say we want to perform the analysis over the same data by district, and compare the feedback of the residents of district 7 with the feedback of the residents of district 10. All we need is our previous KpsResult and the comment_ids of each subset:

In [ ]:
comment_ids_district_7 = list(comments_2016_sample_df[comments_2016_sample_df["Council_District"]==7]["id"].astype(str))
comment_ids_district_10 = list(comments_2016_sample_df[comments_2016_sample_df["Council_District"]==10]["id"].astype(str))

Now, we can compare the full result and the result from each district, using the method **compare_with_comment_subsets()**.
This method receives a dictionary with mappings from subset names to sets of comment ids, and performs the comparison between the full result and the subset results. For the full result and for each of the subsets, you can see the number and percentage of the comments that match each key point. Let's create the comparison df and print the top 20 key points and the summary row.

In [ ]:
subsets_dict = {"district_7":comment_ids_district_7, "district_10":comment_ids_district_10}
comparison_df = kps_result_2016_no_stance.compare_with_comment_subsets(subsets_dict)
pd.concat([comparison_df.head(20),comparison_df.tail(1)])

From the percentage displayed in the comparison, it is apparent that the residents of district 10 are more concerned about the traffic issues in Austin, while the residents of district 7 care more about affordable housing.

Using this method, we can compare results over subsets of the data in the same domain with a single KPS job. The subsets can be data from different GEOs, different organizations, different times, different users (e.g. promoters/detractors) etc.).

This has several advantages over running a separate job for each subset:
- The service only needs to be called once.
- By running a single job, we generate a single key points list which makes it possible to compare between the subsets.
- KPS generates better results for larger datasets, so running over the full data allows to get reacher key points with better coverage. 

### 6.2 Run KPS incrementally 

A year passed, and we collect additional data (from 2017). We would like to analyze the new data and compare it to the data from 2016. 
We can upload all the comments from 2017 to the same domain ("austin_demo").

In [ ]:
comments_2017_df = comments_df[comments_df['year'] == 2017]

sample_size = 1000
comments_2017_sample_df = comments_2017_df.sample(n = sample_size, random_state = 1)

domain = 'austin_demo'
comments_texts_2017 = list(comments_2017_sample_df['text'])
comments_ids_2017 = list(comments_2017_sample_df['id'].astype(str))
keypoints_client.upload_comments(domain=domain, comments_ids=comments_ids_2017, comments_texts=comments_texts_2017)

We can now run a new summarization job over all the data in the domain, as we did before, and automatically extract new key points. Then we can use the **compare_with_comment_subsets** method to compare between the data subsets, as we learned in the previous section. We can assume that some key points will be identical to the key points extracted from the 2016 data, some will be similar and some key points will be new.

A better option is to run a new summarization but provide the keypoints from the 2016 summarization and let *Key Point Summarization* add new key points from the 2017 data if there are such. One benefit of this approach is that the new result will mostly use 2016 key point and we will be consistent with our previous results. Another major benefit for this approach is run-time. 2016 data was already analyzed with these key points and since we have a cache in place, much of the computation can be avoided. The 2016 key points can be provided via the: **run_param['key_point_candidates'] = [...]** parameter, passing a list of strings, or we can use: **run_param['key_point_candidates_by_job_ids'] = [\<job_id1\>,...]** and provide a list of previous job_ids. KPS will take the key points from the jobs' result automatically.

For simplicity, we'll run over the result without the stance analysis. We can also use the incremental approach when running on both stances: we will need to provide the job_id of the positive summarization of 2016 in the run_params_pro and the job_id of negative summarization of 2016 in the run_params_con when running on all sentences from 2016+2017.

First, let's extract the job id from the result:

In [ ]:
stance_to_job_id = kps_result_2016_no_stance.get_stance_to_job_id()
print(stance_to_job_id)
job_id_2016_no_stance = stance_to_job_id["no-stance"]

Now, let's run the job and print the top 20 key points in the comparison df:

In [ ]:
run_params = {'key_point_candidates_by_job_ids': [job_id_2016_no_stance]}
kps_result_2016_2017 = keypoints_client.run_kps_job(domain, run_params=run_params)
subsets_dict = {"2016":comments_ids_2016, "2017":comments_ids_2017}
comparison_df = kps_result_2016_2017.compare_with_comment_subsets(subsets_dict)
pd.concat([comparison_df.head(20),comparison_df.tail(1)])


Alternatively, if you don't care about the 2016+2017 combination and only want to compare the 2016 and the 2017 data, You can use the **comments_ids** parameter in the **run_kps_job** method, to run on a subset of the comments in the domain. Let's do that and run summarization over the comments from 2017 independantly. We will provide the key points from 2016 as candidates, since we want to able to compare between the the two results:

In [ ]:
run_params = {'key_point_candidates_by_job_ids': [job_id_2016_no_stance]}
kps_result_2017_no_stance = keypoints_client.run_kps_job(domain, run_params=run_params, comments_ids = comments_ids_2017)

Now, we can compare the kps_result_2016_no_stance with the kps_result_2017_no_stance using the **compare_with_other_results** method. This method receives the title for the current result and a dictionary mapping from results name to KpsResult objects, and returns the comparison table. If the comparison is with a single other result, the change percent is also displayed. 

In [ ]:
other_results_dict = {"2017": kps_result_2017_no_stance}
comparison_df = kps_result_2016_no_stance.compare_with_other_results(this_title="2016", other_results_dict=other_results_dict)
comparison_df

In the first 63 rows we can see the key points from 2016 applied to the data from 2017. Then, key points from 2017 that are not covered by the 2016 data are added, e.g. *"Focus on basic services"* and *"Austin is not managing growth well."*.

## 7. Running KPS asynchronously

It is also possible to upload comments and run KPS jobs asynchronously. This can be useful when you want to start several jobs simultaneously, and then later collect the results.

Let's create a new domain for sake of the demonstaration. Note that this is not required, as async and sync calls can be used on the same doamin.

In [ ]:
domain = 'austin_demo_async'
keypoints_client.delete_domain_cannot_be_undone(domain=domain)
keypoints_client.create_domain(domain=domain, domain_params={})

### 7.1 Uploading comments asynchronously

In order to start loading comments, use the *upload_comments_async* method: 

In [ ]:
comments_texts = list(comments_2016_sample_df['text'])
comments_ids = list(comments_2016_sample_df['id'].astype(str))
keypoints_client.upload_comments_async(domain=domain, comments_ids=comments_ids, comments_texts=comments_texts)

The method uploads the comments and returns immediately. We must wait until all comments finish processing before starting a KPS job. This can be checked via the **are_all_comments_processed(domain=domain)** method, which prints the upload status and returns True when the domain is ready for running jobs:

In [ ]:
print(keypoints_client.are_all_comments_processed(domain))

You can also use the **wait_till_all_comments_are_processed(domain=domain)** method, that returns only after the comments are processed:

In [ ]:
keypoints_client.wait_till_all_comments_are_processed(domain=domain) 

### 7.2 Running KPS jobs asynchronously

In order to start a job asynchronously, use the **run_kps_job_async** method. This method receives the same arguments as **run_kps_job**, but returns right after a the job is sent to the server, returning a future object:

In [ ]:
future = keypoints_client.run_kps_job_async(domain=domain)

Use the returned future and wait till results are available using the **kps_result = future.get_result()** method. The method waits for the job to finish and eventually returns the result.

In [ ]:
kps_result_async = future.get_result(high_verbosity=True)

The future object can also be used to obtain the job_id, via the **future.get_job_id()** method.

To generate a merged pro and con KpsResult, generate a separate pro_result and con_result using the above flow, and then use the method **KpsResult.get_merged_pro_con_results(pro_result, con_result)**. It's important to only merge pro and con results that were obtained over the same domain and using the same set of comments, otherwise an error will be raised.

In [ ]:
future_pro = keypoints_client.run_kps_job_async(domain, stance="pro")
future_con = keypoints_client.run_kps_job_async(domain, stance="con")
result_pro = future_pro.get_result()
result_con = future_con.get_result()
result_async_merged = KpsResult.get_merged_pro_con_results(pro_result=result_pro, con_result=result_con)

## 8. Cleanup
If you finished the tutorial and no longer need the domains and the results, cleaning up is always advised:

In [ ]:
keypoints_client.delete_domain_cannot_be_undone(domain='austin_demo')
keypoints_client.delete_domain_cannot_be_undone(domain='austin_demo_async')

## 9. Conclusion
In this tutorial, we showed how to use the *Key Point Summarization* service, and how it provides detailed insights over survey data right out of the box - significantly reducing the effort required by a data scientist to analyze the data. We also demonstrated key *key point Summarization* features such as how to modify the summarization parameters and increase coverage, how to use the stance-model and create per-stance results, how to incrementally add new data and how to compare between different subsets of the data.

Feel free to contact us for questions or assistance: *yoavka@il.ibm.com*